# Saving Pots for Bastardo-Jones

## About this notebook

Rose and Oscar want to implement a *Saving Pots* system, whereby we split the money we save each month into pots, one for each area of our lives. So far we are considering
the following pots:
1. New house
2. Current house
3. New and current car
4. Children
5. Holidays
6. Wedding
7. Family
8. Investment

We may add more pots in the future.

We we will spend proportionally to the amount of money we save per month.

This notebook is intended to manage the saving pots.

## Saving accounts and investments
We decided it is best to have actual saving accounts where we can send some money *away*. So far we think two accounts, a higher-yielding one without withdrawals and a
lower-interest one that is easy to withdraw from.

Easy access:
- Current house
- New and current car
- Children
- Holidays
- Family

High-yield:
- New hous  e
- Wedding

Additionally, investment should be sent somewhere else or converted into appreciating assets.

## Technical Challenges
1. How can we maintain the integrity of ground truth data?

## Constants
Some of these values should come from a a csv file with the monthly income, bills and expenses.

In [1]:
import pytest

ROSE_SAVINGS_CURRENT = 23600
OSCAR_SAVINGS_CURRENT = 10000
ROSE_TAKE_HOME_MONTH = 2500
OSCAR_TAKE_HOME_MONTH = 4500
BILLS_EXPENSES_MONTH = 3200
ROSE_POCKET_MONEY_MONTH = 500
OSCAR_POCKET_MONEY_MONTH = 500

SAVING_POTS_CURRENT = ["car", "wedding", "new_house"]
SAVING_POTS_SHARES_CURRENT = {"car": 0.45, "wedding": 0.35, "new_house": 0.20}
assert pytest.approx(sum(SAVING_POTS_SHARES_CURRENT.values())) == 1

SAVING_POTS_MONTH = [
    "new_house",
    "current_house",
    "wedding",
    "holiday",
    "car",
    "investment",
    "children",
    "family",
]
SAVING_POTS_SHARES_MONTH = {
    "new_house": 0.30,
    "current_house": 0.15,
    "holiday": 0.15,
    "wedding": 0.10,
    "car": 0.10,
    "investment": 0.10,
    "children": 0.05,
    "family": 0.05,
}
assert pytest.approx(sum(SAVING_POTS_SHARES_MONTH.values())) == 1

## Back-of-the-envelope projection

In [2]:
import pandas as pd
from datetime import datetime

# current savings as of 03/12/2023
household_current_savings = ROSE_SAVINGS_CURRENT + OSCAR_SAVINGS_CURRENT

# money in a month
household_income_month = ROSE_TAKE_HOME_MONTH + OSCAR_TAKE_HOME_MONTH

# share of expenses
rose_expenses_share = ROSE_TAKE_HOME_MONTH / household_income_month
oscar_expenses_share = OSCAR_TAKE_HOME_MONTH / household_income_month
assert pytest.approx(rose_expenses_share + oscar_expenses_share) == 1


# monthly surplus after paying expenses
rose_surplus_month = ROSE_TAKE_HOME_MONTH - BILLS_EXPENSES_MONTH * rose_expenses_share
oscar_surplus_month = (
    OSCAR_TAKE_HOME_MONTH - BILLS_EXPENSES_MONTH * oscar_expenses_share
)
household_surplus_month = household_income_month - BILLS_EXPENSES_MONTH
assert (
    pytest.approx(rose_surplus_month + oscar_surplus_month) == household_surplus_month
)

# money saved a month
rose_savings_month = rose_surplus_month - ROSE_POCKET_MONEY_MONTH
oscar_savings_month = oscar_surplus_month - OSCAR_POCKET_MONEY_MONTH
household_savings_month = rose_savings_month + oscar_savings_month

# savings projection
savings_projection_df = pd.DataFrame(columns=["month"] + SAVING_POTS_MONTH)
start_month = datetime(2023, 12, 1)
## current savings split
savings_projection_df.loc[0] = [start_month.strftime("%b %y")] + [
    household_current_savings * SAVING_POTS_SHARES_CURRENT[pot]
    if pot in SAVING_POTS_CURRENT
    else 0
    for pot in SAVING_POTS_MONTH
]
## monthly savings projection
for i in range(1, 25):
    month = start_month + pd.DateOffset(months=i)
    savings_projection_df.loc[i] = [month.strftime("%b %y")] + [
        savings_projection_df.loc[i - 1][pot]
        + household_savings_month * SAVING_POTS_SHARES_MONTH[pot]
        for pot in SAVING_POTS_MONTH
    ]
# show savings projection
savings_projection_df

,month,new_house,current_house,wedding,holiday,car,investment,children,family
0,Dec 23,6720.0,0.0,11760.0,0.0,15120.0,0.0,0.0,0.0
1,Jan 24,7560.0,420.0,12040.0,420.0,15400.0,280.0,140.0,140.0
2,Feb 24,8400.0,840.0,12320.0,840.0,15680.0,560.0,280.0,280.0
3,Mar 24,9240.0,1260.0,12600.0,1260.0,15960.0,840.0,420.0,420.0
4,Apr 24,10080.0,1680.0,12880.0,1680.0,16240.0,1120.0,560.0,560.0
5,May 24,10920.0,2100.0,13160.0,2100.0,16520.0,1400.0,700.0,700.0
6,Jun 24,11760.0,2520.0,13440.0,2520.0,16800.0,1680.0,840.0,840.0
7,Jul 24,12600.0,2940.0,13720.0,2940.0,17080.0,1960.0,980.0,980.0
8,Aug 24,13440.0,3360.0,14000.0,3360.0,17360.0,2240.0,1120.0,1120.0
9,Sep 24,14280.0,3780.0,14280.0,3780.0,17640.0,2520.0,1260.0,1260.0


## Split Between Savings Accounts

In [3]:
# split saving pots into high yield and easy access accounts
SAVING_POTS_HIGH_YIELD = ["new_house", "wedding"]
SAVING_POTS_EASY_ACCESS = ["current_house", "car", "children", "holiday", "family"]

# determine monthly contribution to each account type
monthly_contribution_high_yield = (
    sum([SAVING_POTS_SHARES_MONTH[pot] for pot in SAVING_POTS_HIGH_YIELD])
    * household_savings_month
)
monthly_contribution_easy_access = (
    sum([SAVING_POTS_SHARES_MONTH[pot] for pot in SAVING_POTS_EASY_ACCESS])
    * household_savings_month
)

print(
    f"Monthly contribution to high yield accounts: £{monthly_contribution_high_yield:.2f}"
)
print(
    f"Monthly contribution to easy access accounts: £{monthly_contribution_easy_access:.2f}"
)

Monthly contribution to high yield accounts: £1120.00
Monthly contribution to easy access accounts: £1400.00
